In [ ]:
!pip install -q trino

In [ ]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication
from redirect_handler import REDIRECT_HANDLER
import urllib3
import pandas as pd

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TRINO_URI = "https://trino-proxy:443"

# Use the Catalog

Lets setup Anna connection.

1. Open a new private browser window, open `http://localhost:8181` (Lakekeeper UI) and login using Username `anna` and Password `iceberg`. 

2. Execute the following cell - copy the shown login link into the private browser you used before and re-login as `anna` if asked. The cell execution should fail, as Anna has no permissions yet.

3. In your regular browser, navigate to the Warehouse "demo" and grant anna the "select" permission. Now re-run the cell below. Copy the login link to your private browser tab again.

In [ ]:
conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
    catalog="lakekeeper"
)

In [ ]:
cur = conn.cursor()

In [ ]:
# Must return Anna keycloak user id
cur.execute("SELECT current_user").fetchall()

In [ ]:
cur.execute("SHOW CATALOGS").fetchall()

In [ ]:
cur.execute("SHOW TABLES IN customer").fetchall()

In [ ]:
# Execute query and fetch all rows
cur.execute("SELECT * FROM customer.raw_client")

# Data
rows = cur.fetchall()

# Column names
columns = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=columns)

In [ ]:
df.head()

### Try creating/modifying some objects in customer namespace

Anna can only `select` permission on `customer` namespace.

In [ ]:
# Must throw USER ERROR / PERMISSION_DENIED
cur.execute("CREATE TABLE IF NOT EXISTS customer.product (product_id INT, description VARCHAR)")

In [ ]:
# Must throw USER ERROR / PERMISSION_DENIED
cur.execute("ALTER TABLE customer.product DROP COLUMN updated_at")

In [ ]:
# Must throw USER ERROR / PERMISSION_DENIED
cur.execute("""INSERT INTO customer.product 
(id, name, age, category, birth, created_at, updated_at) VALUES 
(1000000, 'Jack', 24, 'adult', 2001, NOW(), NOW())
""")

### Now let Anna create/modify product namespace

"Ask cdo to add create/modify on a new 'product' namespace" -> either go to Management notebook or connect to lakekeeper UI as cdo and create product namespace + assign permissions to Anna

In [ ]:
cur.execute("SELECT current_user").fetchall()

In [ ]:
cur.execute("GRANT ALL PRIVILEGES ON TABLE product.enriched_product TO USER \"cfb55bf6-fcbb-4a1e-bfec-30c6649b52f8\"")

In [ ]:
# Should succeed
cur.execute("CREATE TABLE IF NOT EXISTS product.raw_product (id INT, description VARCHAR, price DOUBLE)")

In [ ]:
cur.execute("""INSERT INTO product.raw_product 
(0, 'Product 1', 8.95),
(1, 'Product 2', 17.95),
(2, 'Product 3', 10)
""")

In [ ]:
# Make sure data has been written
cur.execute("SELECT * FROM product.raw_product").fetchall()

#### Create another table that Peter cannot access

In [ ]:
# Should succeed
cur.execute("CREATE TABLE IF NOT EXISTS product.enriched_product (id INT, description VARCHAR, price DOUBLE, cost DOUBLE)")

In [ ]:
cur.execute("""INSERT INTO product.enriched_product 
(0, 'Product 1', 8.95, 4.12),
(1, 'Product 2', 17.95, 9.28),
(2, 'Product 3', 10, 3.02)
""")

In [ ]:
# Make sure data has been written
cur.execute("SELECT * FROM product.enriched_product").fetchall()